In [9]:
from keras.models import model_from_json
from keras.preprocessing import image
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
from openpyxl import load_workbook
import pandas as pd
from playsound import playsound
#######################################Face Recognition###############################################

#Take the images from file and encoding them: 
path = 'faces'
images = []
classNames = []
myList = os.listdir(path)

for cl in myList:                                #Looping through the file and take the images.
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])  # split the name of person from the image name
print('Students list')
print(classNames)  #print all names

#images encoding:
encodeList = []
for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = face_recognition.face_encodings(img)[0]
    encodeList.append(encode)
encodeListKnown =encodeList

######################################################################################################
#############################Drowsiness_Detection#####################################################
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
flag=0
#####################################Emotion analysis#################################################
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

model = model_from_json(open("model.json", "r").read())
#load weights
model.load_weights('model.h5')
################################################Emotion Detection#######################################################    
#open webcam:
cap = cv2.VideoCapture(0)
 
while True:  
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (400, 400))
    cv2.imshow('Facial emotion analysis ',resized_img)
    

#######################################Face Recognition###############################################


    success, img = cap.read()  #take captures from webcam
  
    #find faces from webcam and compare it with our images in file:
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis) # to find best match
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            
            #to determine the Faces by rectungle with names in the bottom:
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            
            # Record the attendence in Excel:
            with open('Attendance_List1.csv','r+') as f:
                myDataList = f.readlines()
                nameList = []
                for line in myDataList:
                    entry = line.split(',')
                    nameList.append(entry[0])
                if name not in nameList:
                    now = datetime.now()
                    dtString = now.strftime('%d/%b/%Y, %H:%M:%S')
                        #f.writelines(f'\n{name},{dtString}')
                    today8am = now.replace(hour=7, minute=0, second=0, microsecond=0)
                    if now < today8am:
                        f.writelines(f'\n{name},{dtString},NO,{predicted_emotion}')
                    else:
                        f.writelines(f'\n{name},{dtString},YES,{predicted_emotion}')
   
                      
    cv2.imshow('Attendence',img)

#############################Drowsiness_Detection#####################################################

    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)


        if ear < thresh:
            flag += 1

            if flag >= frame_check:               
                cv2.putText(frame, "****************Wake Up!****************", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "****************Wake Up!****************", (10,325),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                wavFile = "alarm.wav "
                playsound(wavFile)
        else:
            flag = 0
    cv2.imshow("Drowsniss Alert", frame)
    # close window when Esc button pressed:
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break
cap.release()
cv2.destroyAllWindows()



Students list
['Abdulrahman Alamer', 'Hussain Alobaidi', 'Mohammed Alshamasi', 'mustafa alolaiwat']
